# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [2]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.7 MB/s eta 0:00:0

In [3]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [7]:
# If you're using colab, run this
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [9]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [28]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [30]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Specify a persist_directory for Chroma to store the database
persist_directory = "chroma_db"

docsearch = Chroma.from_documents(
    texts,
    embeddings,
    collection_name="sonnets",
    persist_directory=persist_directory  # Pass the persist_directory to Chroma
)

In [31]:
Sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [13]:
from langchain_community.document_loaders import WebBaseLoader

In [42]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [44]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
# Specify the persist_directory when creating the Chroma instance
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff", persist_directory="chroma_db")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [16]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [45]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=Sonnets.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [46]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [47]:
agent.invoke(
    "What themes are prevalent in Shakespeare's sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What themes are prevalent in Shakespeare's sonnets?"
Observation:  The themes prevalent in Shakespeare's sonnets include the fleeting nature of beauty, the power of time and mortality, the struggle between love and reason, and the immortality of art and poetry. Other common themes include the power of love, the beauty of nature, and the complexities of human relationships.
Thought: I now know the final answer.
Final Answer: The themes prevalent in Shakespeare's sonnets include the fleeting nature of beauty, the power of time and mortality, the struggle between love and reason, and the immortality of art and poetry. Other common themes include the power of love, the beauty of nature, and the complexities of human relationships.

> Finished chain.


{'input': "What themes are prevalent in Shakespeare's sonnets?",
 'output': "The themes prevalent in Shakespeare's sonnets include the fleeting nature of beauty, the power of time and mortality, the struggle between love and reason, and the immortality of art and poetry. Other common themes include the power of love, the beauty of nature, and the complexities of human relationships."}

In [48]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the pros and cons of each tool before making a decision.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff has the ability to automatically fix some of its own lint violations, while Flake8 does not have this feature.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5. Can be used as a formatter: Ruff can be used as a form

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff may be a better choice for those looking for a more comprehensive and efficient linter, with a larger rule set, better compatibility with other tools, automatic fixing of lint violations, and the ability to be used as a formatter.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [49]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=Sonnets.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [50]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [51]:
agent.invoke(
    "What is the central idea of Sonnet 116?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to find the central idea.
Action: Sonnets QA System
Action Input: "What is the central idea of Sonnet 116?"
Observation:  The central idea of Sonnet 116 is that true love is constant and unchanging, even in the face of challenges and the passing of time.


> Finished chain.


{'input': 'What is the central idea of Sonnet 116?',
 'output': ' The central idea of Sonnet 116 is that true love is constant and unchanging, even in the face of challenges and the passing of time.'}

In [52]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the pros and cons of each tool before making a decision.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5. Can be used as a formatter: Ruff can be used as a formatter, 

{'input': 'Why use ruff over flake8?',
 'output': '\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.\n\n3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.\n\n5. Can be used as a formatter: Ruff can be used as a formatter, making it a more versatile tool than Flake8.\n\nUltimately, the choice between Ruff and Flake8 will depend on the specific needs and preferences

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [53]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=Sonnets.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [54]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [55]:
agent.invoke(
    "What is the tone of Sonnet 130?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What is the tone of Sonnet 130?"
Observation:  The tone of Sonnet 130 is satirical and mocking.
Thought: I should also check the Ruff QA System to see if it has any additional insights.
Action: Ruff QA System
Action Input: "What is the tone of Sonnet 130?"
Observation:  I don't know.
Thought: I should try rephrasing the question to see if I can get a better answer.
Action: Sonnets QA System
Action Input: "How would you describe the tone of Sonnet 130?"
Observation:  The tone of Sonnet 130 is playful and satirical. The speaker is mocking the traditional conventions of love poetry and challenging the idea of idealized beauty.
Thought: I now know the final answer.
Final Answer: The tone of Sonnet 130 is playful and satirical. The speaker is mocking the traditional conventions of love poetry and challenging the idea of idealized beauty.

> Finished c

{'input': 'What is the tone of Sonnet 130?',
 'output': 'The tone of Sonnet 130 is playful and satirical. The speaker is mocking the traditional conventions of love poetry and challenging the idea of idealized beauty.'}